<a href="https://colab.research.google.com/github/jaygo-kr/brunch/blob/master/03_04_summarize_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 드라마 대본 요약

* 이전 노트에서 정제한 대본 자료의 문장들로 대본을 요약해(100자 이내) 보도록 하자

In [212]:
import os
import numpy as np
import pandas as pd
from google.colab import files, drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
DATA_PATH = './gdrive/My Drive/share/suwon study/script/JAY완료/' # 대본 데이터
TRAIN_PATH = './gdrive/My Drive/share/suwon study/sentiment/' # 네이버 영화 리뷰 감성데이터

In [214]:
os.listdir(DATA_PATH)[:5]

['W+제11회+대본.csv',
 'W+제12회+대본.csv',
 'W+제13회+대본.csv',
 'W+제15회+대본.csv',
 'W+제4회+대본.csv']

In [215]:
os.listdir(TRAIN_PATH)

['ratings.txt',
 'ratings_test.txt',
 'ratings_train.txt',
 'train.csv',
 'test.csv',
 'script.csv']

In [0]:
script_df = pd.read_csv(DATA_PATH + 'W+제1회+대본.csv')

In [217]:
script_df.head()

,act,content
0,씬/1,아테네 올림픽 자료화면 인서트 \n
1,지문,자막 – 2004년 8월. 아테네 올림픽\n한국 국가대표 경기 생중계 자료화면.\n...
2,씬/2,"방송 스튜디오 (과거, 밤)\n"
3,지문,올림픽 중계 스튜디오. 아테네 올림픽 엠블럼과 마스코트를 배경으로 남자 MC가 진행...
4,MC,"네, 잠시 종목을 바꿔보겠습니다. 지금 사격 경기장에서는 50미터 권총 남자 결선이..."


## **LSTM 모델**
* Keras 창시자에게 배우는 딥러닝 책에 나오는 LSTM 문장생성 모델 적용
* 해당 모델은 음절단위 정보를 활용한 모델이다

### 전처리

* 대본 정리

In [0]:
script_df.content = script_df.content.map(lambda x: x.rstrip('\n')) # 개행문자 제거

In [0]:
# 대사에만 집중하기 위해 지문, 씬, C#등 제거 - '지문', '씬/.*', 'C#.*'
script_df_actor = script_df.drop(script_df[script_df['act'].str.find('지문') == 0].index)
script_df_actor = script_df_actor.drop(script_df_actor[script_df_actor['act'].str.find('씬/') == 0].index)
script_df_actor = script_df_actor.drop(script_df_actor[script_df_actor['act'].str.find('C#') == 0].index)

In [220]:
# 전처리 누락분 제거
script_df_actor[script_df_actor['act']=='  ']

,act,content
373,,"지글지글 고기 굽는 소리, 칼질하는 소리, 매운 연기에 라운지에서 들려오는 음악소리..."


In [0]:
# 대사들을 학습하기 위해 연결
content_tie_str = script_df_actor.content.str.cat(sep=' ')

In [222]:
print(content_tie_str[:100])

네, 잠시 종목을 바꿔보겠습니다. 지금 사격 경기장에서는 50미터 권총 남자 결선이 펼쳐지고 있는데요. 대한민국의 강철 선수가 예상 밖 선전 중이라고 합니다. 강철 선수는 이제 겨


In [223]:
len(content_tie_str)

14737

In [224]:
# 한글 태깅을 위한 패키지 설치
! pip install konlpy
from konlpy.tag import Okt
tagger = Okt()

In [0]:
content_tie_pos = tagger.pos(content_tie_str, norm=True)

In [226]:
content_tie_pos[:10]

[('네', 'Noun'),
 (',', 'Punctuation'),
 ('잠시', 'Noun'),
 ('종목', 'Noun'),
 ('을', 'Josa'),
 ('바꿔', 'Verb'),
 ('보겠습니다', 'Verb'),
 ('.', 'Punctuation'),
 ('지금', 'Noun'),
 ('사격', 'Noun')]

In [0]:
def pos_inv(pos):
  return pos[0]

In [0]:
#content_tie = list(map(pos_inv, content_tie_pos))
content_tie = content_tie_str

In [229]:
len(content_tie)

14737

* 원핫 인코딩

In [230]:
maxlen = 100
step = 2

sentences = []

next_chars = []

for i in range(0, len(content_tie) - maxlen, step):
  sentences.append(content_tie[i:i + maxlen])
  next_chars.append(content_tie[i + maxlen])
print(len(sentences))

7319


In [231]:
pd.DataFrame({'sentences':sentences, 'next_chars':next_chars})

,sentences,next_chars
0,"네, 잠시 종목을 바꿔보겠습니다. 지금 사격 경기장에서는 50미터 권총 남자 결선이...",우
1,잠시 종목을 바꿔보겠습니다. 지금 사격 경기장에서는 50미터 권총 남자 결선이 펼...,고
2,시 종목을 바꿔보겠습니다. 지금 사격 경기장에서는 50미터 권총 남자 결선이 펼쳐지...,학
3,종목을 바꿔보겠습니다. 지금 사격 경기장에서는 50미터 권총 남자 결선이 펼쳐지고 ...,
4,을 바꿔보겠습니다. 지금 사격 경기장에서는 50미터 권총 남자 결선이 펼쳐지고 있는...,학
...,...,...
7314,! 내가 봤다고! (E) 누나 좀 진정하고.. 난 뭔 소리를 하는 건지 (살아있는 ...,키
7315,내가 봤다고! (E) 누나 좀 진정하고.. 난 뭔 소리를 하는 건지 (살아있는 사람...,
7316,봤다고! (E) 누나 좀 진정하고.. 난 뭔 소리를 하는 건지 (살아있는 사람에게...,는
7317,다고! (E) 누나 좀 진정하고.. 난 뭔 소리를 하는 건지 (살아있는 사람에게 말...,.


In [232]:
chars = sorted(list(set(content_tie))) # 중복없는 글자 리스트
print(len(chars))

701


In [0]:
char_indices = dict((char, chars.index(char)) for char in chars)

In [0]:
# char_indices

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '(': 4,
 ')': 5,
 ',': 6,
 '.': 7,
 '/': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '4': 13,
 '5': 14,
 ...
}

In [0]:
# 문자을 위 인덱스를 바탕으로 벡터화 하여 학습 가능하도록 x,y로 생성
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
  y[i, char_indices[next_chars[i]]] = 1

In [236]:
x.shape, y.shape

((7319, 100, 701), (7319, 701))

In [237]:
x[:3]

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False,  True, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False,  True, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False,  True, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False,  True, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False,  True, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False,  True, ..., False, False, False],
        [False, False, False, ..., False, Fal

### 모델링

In [0]:
from keras import layers, models

model = models.Sequential()
model.add(layers.LSTM(128, input_shape = (maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [0]:
model.compile(optimizer = 'rmsprop',
             loss = 'categorical_crossentropy')

In [0]:
def sample(preds, temperature = 0.1):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds/ np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1) # 주사위 던지기 1번 1차원으로, 그래서 몇번 나왔는가
  return np.argmax(probas)

In [241]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(content_tie) - maxlen -1)

epoch = 20
print("epochs : ", epoch)
model.fit(x, y, batch_size=128, epochs=epoch)

seed_text = content_tie[start_index : start_index + maxlen]
print("seed text : ", seed_text)

for temperature in [0.2, 0.5, 1.0, 1.2]:
  print("temp : ", temperature)
  generated_text = seed_text
  full_text = ""
  print(generated_text)
  
  for i in range(100):
    sampled = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text):
      sampled[0, t, char_indices[char]] = 1.
      
    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = chars[next_index]

    generated_text += next_char # 덧셈하면 맨위에 붙음
    generated_text = generated_text[1:] # 앞에 자름
    full_text += next_char

    #sys.stdout.write(next_char)
    #sys.stdout.flush()
    
  print("generated text : ", full_text)

epochs :  20
Epoch 1/20
7319/7319 [==============================] - 11s 2ms/step - loss: 4.8666
Epoch 2/20
7319/7319 [==============================] - 10s 1ms/step - loss: 4.5960
Epoch 3/20
7319/7319 [==============================] - 10s 1ms/step - loss: 4.5530
Epoch 4/20
7319/7319 [==============================] - 10s 1ms/step - loss: 4.4814
Epoch 5/20
7319/7319 [==============================] - 10s 1ms/step - loss: 4.3699
Epoch 6/20
7319/7319 [==============================] - 10s 1ms/step - loss: 4.2558
Epoch 7/20
7319/7319 [==============================] - 10s 1ms/step - loss: 4.1525
Epoch 8/20
7319/7319 [==============================] - 10s 1ms/step - loss: 4.0639
Epoch 9/20
7319/7319 [==============================] - 11s 1ms/step - loss: 3.9809
Epoch 10/20
7319/7319 [==============================] - 10s 1ms/step - loss: 3.9114
Epoch 11/20
7319/7319 [==============================] - 10s 1ms/step - loss: 3.8390
Epoch 12/20
7319/7319 [==============================] - 10s 

In [242]:
full_text

' 계줍튼끝오들람어네. 말 보대지  일어요.. 더생터 묘예서? 히. 얼어도갑세달둬진의 보? 지앞면어나 전이스된 년셨정적기 많광서선신... 이라요. 네을 우기지이 터히어 자 번나신반 '

> 알파벳과는 다르게 한글 글자의 종류가 무궁무진하여 단어 기반으로 하는 LSTM 모델은 문장을 생성하는데 부적합 하다.

### **bigram 모델**
* 해당 모델은 ratsgo 님의 블로그를 참조한 모델이다
* 어절을 기본단위로 하여 분석하는 모델이다

### 전처리

In [0]:
script_df_actor.content = script_df_actor.content.map(lambda x : x.split(' ')) # 어절단위 분해

In [0]:
from collections import defaultdict

In [0]:
bigram_transitions = defaultdict(list)

In [246]:
bigram_transitions

defaultdict(list, {})

In [0]:
for line in script_df_actor.content:
  for prev, current in zip(line, line[1:]):
    bigram_transitions[prev].append(current)

In [0]:
#print(bigram_transitions)

defaultdict(<class 'list'>, {'네,': ['잠시', '여기는'], '잠시': ['종목을', '쳐다보며', '후)'], ... })

In [0]:
import random

def generate_using_bigrams(cut=30):
  idx = 0
  stp = 0
  result = []
  while True:
    current = random.choice(list(bigram_transitions.keys()))
    if bigram_transitions[current] != []:
      break

  while True:
      next_word_candidates = bigram_transitions[current]

      while True:
        stp += 1
        current = random.choice(next_word_candidates)
        if bigram_transitions[current] != []:
          break
          
        if stp == 300:
          return print("Over run")
           
      result.append(current)
      if idx == cut:
          return " ".join(result)
      idx += 1

In [250]:
generate_using_bigrams()

'경찰은 강대표를 걱정해 모인 시민들로 발 중 당일 행적에 많은 의문점을 발견한 것으로 알려졌습니다.                 '

In [251]:
generate_using_bigrams()

Over run


In [252]:
generate_using_bigrams()

Over run


In [253]:
generate_using_bigrams()

Over run


> bigram을 위해서는 충분한 defalutdict 가 구성되어야 하는데, 지금 주어진 대본만으로는 하나의 단어에 연결된 biragm 단어가 1개인 경우가 많아서 기존에 존재하는 문장을 그대로 산출하게 되거나, 문장을 종결하게 된다.